# Part 1. PicoCTF

Names: Mateo Fuertes (00321987), Melisa Guerrero (00322205), Joel Cuascota (00327494)

Course: CMP 5006 - Information Security (NRC: 1230)

Institution: Universidad San Francisco de Quito

## The numbers

### First Technique: Manual Decryption and Execution (Shift)

In [ ]:
# List of numbers representing the letters 16, 9, 3, 15, 3, 20, 6{20, 8, 5, 14, 21, 13, 2, 5, 18, 19, 13, 1, 19, 15, 14}
numbers = [16, 9, 3, 15, 3, 20, 6]
key_numbers = [20, 8, 5, 14, 21, 13, 2, 5, 18, 19, 13, 1, 19, 15, 14]

# Function to convert numbers into letters
def numbers_to_letters(numbers):
    return ''.join([chr(64 + num) for num in numbers])

# Decode the lists
message = numbers_to_letters(numbers)
key_message = numbers_to_letters(key_numbers)

# Print the result
print(message, end='')
print(f"{{{key_message}}}")


PICOCTF{THENUMBERSMASON}


### Second Technique: Automating the Entire Process (Caesar Cypher)

In [78]:
def string_to_numbers(string):
    pico_numbers = []
    flag_numbers = []
    string_numbers = string.split("{")
    pico_numbers = [int(x) for x in string_numbers[0].split(", ")]
    flag_numbers = [int(x) for x in string_numbers[1].replace("}", "").split(", ")]
    return pico_numbers, flag_numbers

def find_shift(numbers):
    original_text = "picoCTF"
    original_numbers = [ord(c) - ord('A') + 1 if c.isupper() else ord(c) - ord('a') + 1 for c in original_text]
    shift = (numbers[0] - original_numbers[0]) % 26
    return shift

def caesar_decypher(numbers, shift):
    text = ""
    for num in numbers:
        shifted_num = (num - shift - 1) % 26 + 1
        text += chr(shifted_num + ord('A') - 1)
    return text.lower()

def main():
    numbers_string = "16, 9, 3, 15, 3, 20, 6{20, 8, 5, 14, 21, 13, 2, 5, 18, 19, 13, 1, 19, 15, 14}"
    pico_numbers, flag_numbers = string_to_numbers(numbers_string)
    shift = find_shift(pico_numbers)
    text = caesar_decypher(flag_numbers, shift)
    print("Text: picoCTF{", text, "}", sep="")

if __name__ == '__main__':
    main()

Text: picoCTF{thenumbersmason}


## C3

### First Technique: Manual Decryption and Execution

Invert the convert.py code for decrypting the message

In [ ]:
import sys
chars = ""
from fileinput import input
for line in input():
  chars += line

lookup1 = "\n \"#()*+/1:=[]abcdefghijklmnopqrstuvwxyz"
lookup2 = "ABCDEFGHIJKLMNOPQRSTabcdefghijklmnopqrst"

out = ""
prev = 0

for char in chars:
    if char in lookup2:
        cur = lookup2.index(char)
        original_index = (cur + prev) % 40
        out += lookup1[original_index]
        prev = original_index
    else:
        out += char 

sys.stdout.write(out)

Run the code obtained (with a few Python3-compatible modifications) with the file itself as an input:

cat file.py | python3 file2.py

In [ ]:
#asciiorder
#fortychars
#selfinput
#pythonthree

chars = ""
from fileinput import input
for line in input():
    chars += line
b = 1 // 1

for i in range(len(chars)):
    if i == b * b * b:
        print(chars[i])
        b += 1 // 1 


Result: adlibs

### Second Technique: Server Automation via Telnet

In [79]:
import sys
from itertools import cycle

def affine_decrypt(text):
    lookup1 = "\n \"#()*+/1:=[]abcdefghijklmnopqrstuvwxyz"
    lookup2 = "ABCDEFGHIJKLMNOPQRSTabcdefghijklmnopqrst"
    
    out = ""
    prev = 0
    
    for char in text:
        if char in lookup2:
            cur = lookup2.index(char)
            original_index = (cur + prev) % 40
            out += lookup1[original_index]
            prev = original_index
        else:
            out += char 
    
    return out

def extract_hidden_message(ciphertext):
    result = []
    indices = [n**3 for n in range(1, int(len(ciphertext) ** (1/3)) + 2)]
    
    for idx in indices:
        if idx < len(ciphertext):
            result.append(ciphertext[idx])
    
    return ''.join(result)

def main():
    ciphertext = "DLSeGAGDgBNJDQJDCFSFnRBIDjgHoDFCFtHDgJpiHtGDmMAQFnRBJKkBAsTMrsPSDDnEFCFtIbEDtDCIbFCFtHTJDKerFldbFObFCFtLBFkBAAAPFnRBJGEkerFlcPgKkImHnIlATJDKbTbFOkdNnsgbnJRMFnRBNAFkBAAAbrcbTKAkOgFpOgFpOpkBAAAAAAAiClFGIPFnRBaKliCgClFGtIBAAAAAAAOgGEkImHnIl"  # Replace with actual encrypted text
    
    # Step 1: Encrypt the input text
    encrypted_text = affine_decrypt(ciphertext)
    print("Encrypted Text:", encrypted_text)
    
    # Step 2: Extract hidden message from encrypted text
    hidden_message = extract_hidden_message(encrypted_text)
    print("Extracted Hidden Message:", hidden_message)

if __name__ == "__main__":
    main()

Encrypted Text: #asciiorder
#fortychars
#selfinput
#pythontwo

chars = ""
from fileinput import input
for line in input():
    chars += line
b = 1 / 1

for i in range(len(chars)):
    if i == b * b * b:
        print chars[i] #prints
        b += 1 / 1

Extracted Hidden Message: adlibs


## rsa_oracle

In this problem, two different techniques are provided to show how the same task can be solved using distinct approaches. The first method leverages pwntools—a specialized CTF library that simplifies remote interactions with high-level functions and abstractions. The second method uses telnetlib from Python's standard library, requiring more manual handling of the communication but offering a viable alternative without external dependencies. This demonstrates the versatility in problem-solving and satisfies the requirement to present two distinct techniques for solving the same problem.

In [ ]:
%pip install pwn

### First Technique: Server Interaction via Pwn Library

In [75]:
from pwn import remote

def rsa_oracle_attack():
    # Set up connection
    host, port = "titan.picoctf.net", 54232
    p = remote(host, port)
    p.recvuntil(b"decrypt.")
    
    # Read the encrypted password from the file
    with open("/Users/melisaguerrero/Downloads/password.enc") as file:
        c = int(file.read())
    
    # Request encryption of 2
    p.sendline(b"E")
    p.recvuntil(b"keysize): ")
    p.sendline(b"\x02")
    p.recvuntil(b"mod n) ")
    
    # Read the encryption of 2
    c_a = int(p.recvline())
    
    # Request decryption of (2 * encrypted password)
    p.sendline(b"D")
    p.recvuntil(b"decrypt: ")
    p.sendline(str(c_a * c).encode())
    p.recvuntil(b"mod n): ")
    
    # Extract the decrypted password and divide by 2 to get the original
    password = int(p.recvline(), 16) // 2
    password = password.to_bytes(len(str(password))-7, "big").decode("utf-8")
    
    print("Recovered Password:", password)
    
    p.close()

def main():
    rsa_oracle_attack()

if __name__ == '__main__':
    main()

Recovered Password: 881d9


### Second Technique: Server Automation via Telnet

In [83]:
import telnetlib

HOST = "titan.picoctf.net"
PORT = 54232

# Connect to the server using telnetlib.
tn = telnetlib.Telnet(HOST, PORT)

# Wait until we receive the banner that includes "decrypt."
banner = tn.read_until(b"decrypt.")
print("Initial Banner:\n", banner.decode(errors="ignore"))

# Read the encrypted content from the file.
with open("/Users/melisaguerrero/Downloads/password.enc") as file:
    c = int(file.read())

# Send the "E" command and wait for the prompt.
tn.write(b"E\n")
tn.read_until(b"keysize): ")
# Send the keysize (in this case, the byte value 2).
tn.write(b"\x02\n")
tn.read_until(b"mod n) ")

# Receive the line with c_a and convert it to an integer.
c_a_line = tn.read_until(b"\n").strip()
c_a = int(c_a_line)

# Send the "D" command for decryption.
tn.write(b"D\n")
tn.read_until(b"decrypt: ")
# Send the product of c_a and c for decryption.
tn.write((str(c_a * c) + "\n").encode())

# Wait until the final prompt ("mod n): ") is received.
tn.read_until(b"mod n): ")

# Read the final response line.
result_line = tn.read_until(b"\n").strip()

# Process the response: divide the integer (parsed from hex) by 2,
# then convert it to bytes using the original formula.
password_int = int(result_line, 16) // 2
password = password_int.to_bytes(len(str(password_int)) - 7, "big").decode("utf-8")

print("Password:", password)
tn.close()


Initial Banner:
 *****************************************
****************THE ORACLE***************
*****************************************
what should we do for you? 
E --> encrypt D --> decrypt.
Password: 881d9


Run this command in the terminal:

openssl enc -aes-256-cbc -d -in secret.enc

## interencdec

### First Technique: Caesar Cipher Brute-Force

In [26]:
import base64

# Base64 encoded text
encoded_text = "YidkM0JxZGtwQlRYdHFhR3g2YUhsZmF6TnFlVGwzWVROclgyMHdNakV5TnpVNGZRPT0nCg=="

# First Base64 decoding: decode the encoded text and convert it to a UTF-8 string,
# ignoring any decoding errors.
decoded_text_1 = base64.b64decode(encoded_text).decode("utf-8", errors="ignore")

print("First decoding:", decoded_text_1)




First decoding: b'd3BqdkpBTXtqaGx6aHlfazNqeTl3YTNrX20wMjEyNzU4fQ=='



In [28]:
# The first decoding produced this Base64 string:
decoded_text_1 = 'd3BqdkpBTXtqaGx6aHlfazNqeTl3YTNrX20wMjEyNzU4fQ=='

# Second Base64 decoding: decode the string from the first decoding step,
# then convert it to a UTF-8 string, ignoring any decoding errors.
decoded_text_2 = base64.b64decode(decoded_text_1).decode("utf-8", errors="ignore")
print("Second decoding:", decoded_text_2)



Second decoding: wpjvJAM{jhlzhy_k3jy9wa3k_m0212758}


In [36]:
# Encrypted text
cipher_text = "wpjvJAM{jhlzhy_k3jy9wa3k_m0212758}"

def caesar_cipher(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            # Set base for upper or lower case
            base = ord('A') if char.isupper() else ord('a')
            result += chr((ord(char) - base - shift) % 26 + base)
        else:
            # Keep digits and symbols unchanged
            result += char
    return result

def brute_force_caesar(text):
    possible_texts = {}
    for shift in range(26):  # Try all shifts
        possible_texts[shift] = caesar_cipher(text, shift)
    return possible_texts

# Run brute force decryption
brute_force_results = brute_force_caesar(cipher_text)

brute_force_results


{0: 'wpjvJAM{jhlzhy_k3jy9wa3k_m0212758}',
 1: 'voiuIZL{igkygx_j3ix9vz3j_l0212758}',
 2: 'unhtHYK{hfjxfw_i3hw9uy3i_k0212758}',
 3: 'tmgsGXJ{geiwev_h3gv9tx3h_j0212758}',
 4: 'slfrFWI{fdhvdu_g3fu9sw3g_i0212758}',
 5: 'rkeqEVH{ecguct_f3et9rv3f_h0212758}',
 6: 'qjdpDUG{dbftbs_e3ds9qu3e_g0212758}',
 7: 'picoCTF{caesar_d3cr9pt3d_f0212758}',
 8: 'ohbnBSE{bzdrzq_c3bq9os3c_e0212758}',
 9: 'ngamARD{aycqyp_b3ap9nr3b_d0212758}',
 10: 'mfzlZQC{zxbpxo_a3zo9mq3a_c0212758}',
 11: 'leykYPB{ywaown_z3yn9lp3z_b0212758}',
 12: 'kdxjXOA{xvznvm_y3xm9ko3y_a0212758}',
 13: 'jcwiWNZ{wuymul_x3wl9jn3x_z0212758}',
 14: 'ibvhVMY{vtxltk_w3vk9im3w_y0212758}',
 15: 'haugULX{uswksj_v3uj9hl3v_x0212758}',
 16: 'gztfTKW{trvjri_u3ti9gk3u_w0212758}',
 17: 'fyseSJV{squiqh_t3sh9fj3t_v0212758}',
 18: 'exrdRIU{rpthpg_s3rg9ei3s_u0212758}',
 19: 'dwqcQHT{qosgof_r3qf9dh3r_t0212758}',
 20: 'cvpbPGS{pnrfne_q3pe9cg3q_s0212758}',
 21: 'buoaOFR{omqemd_p3od9bf3p_r0212758}',
 22: 'atnzNEQ{nlpdlc_o3nc9ae3o_q0212758}',
 23: 'zsmyMDP{mkockb_

When shift = 7, we obtain the plaintext: 'picoCTF{caesar_d3cr9pt3d_f0212758}'.

### Second Technique: Affine Cipher Brute-Force Decryption

In [ ]:
# Function to calculate the modular inverse of 'a' in Z26
def modinv(a, m):
    """Calculates the modular inverse of 'a' modulo 'm', if it exists."""
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

# Function to decrypt using the affine cipher
def affine_decrypt(cipher_text, a, b):
    """
    Decrypts the given cipher text using the affine cipher with key (a, b).
    The decryption function is:
      d(y) = a_inv * (y - b) mod 26
    where a_inv is the modular inverse of 'a' in Z26.
    """
    a_inv = modinv(a, 26)
    if a_inv is None:
        return None  # If 'a' is not invertible, skip this attempt
    
    decrypted = ""
    for char in cipher_text:
        if char.isalpha():
            is_upper = char.isupper()
            base = ord('A') if is_upper else ord('a')
            y = ord(char) - base
            x = (a_inv * (y - b)) % 26
            decrypted += chr(x + base)
        else:
            decrypted += char 
    return decrypted

# Provided cipher text
cipher_text = "wpjvJAM{jhlzhy_k3jy9wa3k_m0212758}"

# Test all possible combinations of (a, b) where 'a' is invertible in Z26
valid_a_values = [a for a in range(1, 26) if modinv(a, 26) is not None]
possible_decryptions = {}

# Try all valid combinations of (a, b) and decrypt the text
for a in valid_a_values:
    for b in range(26):
        decrypted_text = affine_decrypt(cipher_text, a, b)
        if decrypted_text:
            possible_decryptions[(a, b)] = decrypted_text

import pandas as pd

# Create a DataFrame with the results
df_results = pd.DataFrame(possible_decryptions.items(), columns=["(a, b)", "Decrypted Text"])

# Display the first 10 rows of the results
df_results.head(10)


,"(a, b)",Decrypted Text
0,"(1, 0)",wpjvJAM{jhlzhy_k3jy9wa3k_m0212758}
1,"(1, 1)",voiuIZL{igkygx_j3ix9vz3j_l0212758}
2,"(1, 2)",unhtHYK{hfjxfw_i3hw9uy3i_k0212758}
3,"(1, 3)",tmgsGXJ{geiwev_h3gv9tx3h_j0212758}
4,"(1, 4)",slfrFWI{fdhvdu_g3fu9sw3g_i0212758}
5,"(1, 5)",rkeqEVH{ecguct_f3et9rv3f_h0212758}
6,"(1, 6)",qjdpDUG{dbftbs_e3ds9qu3e_g0212758}
7,"(1, 7)",picoCTF{caesar_d3cr9pt3d_f0212758}
8,"(1, 8)",ohbnBSE{bzdrzq_c3bq9os3c_e0212758}
9,"(1, 9)",ngamARD{aycqyp_b3ap9nr3b_d0212758}


## HideToSee



### First Technique: Steghide and Atbash

Steghide is a tool used to embed and extract this data, securing it with a password. To recover the hidden data without knowing the password.

#### 1. Create the Dockerfile

Open your terminal and run the following command:

```bash
nano Dockerfile




#### Dockerfile Content

```dockerfile
FROM debian:latest

# Update package lists and install steghide
RUN apt update && apt install -y steghide

# Set the working directory to /data
WORKDIR /data

# Set steghide as the default command
ENTRYPOINT ["steghide"]
```text


#### 3. Save the file

#### 4. Build the Docker image
```bash
docker build -t steghide-container .




#### 5. Run Steghide in Docker
```bash
(base) melisaguerrero@MacBook-Pro-de-Melisa-2 ~ % docker run --rm -v /Users/melisaguerrero/Downloads:/data -it steghide-container extract -sf /data/atbash.jpg -xf /data/encrypted.txt
Enter passphrase: 
the file "/data/encrypted.txt" does already exist. overwrite ? (y/n) y
wrote extracted data to "/data/encrypted.txt".
(base) melisaguerrero@MacBook-Pro-de-Melisa-2 ~ % cat /Users/melisaguerrero/Downloads/encrypted.txt
krxlXGU{zgyzhs_xizxp_8z0uvwwx}

![Resultado de DCode](dcode.png)

In [61]:
def atbash_cipher(text):
    alphabet_lower = 'abcdefghijklmnopqrstuvwxyz'
    alphabet_upper = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    # create distionary
    atbash_lower = {alphabet_lower[i]: alphabet_lower[-i-1] for i in range(len(alphabet_lower))}
    atbash_upper = {alphabet_upper[i]: alphabet_upper[-i-1] for i in range(len(alphabet_upper))}

    # substitution
    result = []
    for char in text:
        if char.islower():
            result.append(atbash_lower.get(char, char))
        elif char.isupper():
            result.append(atbash_upper.get(char, char))
        else:
            result.append(char)
    
    return ''.join(result)


text = "krxlXGU{zgyzhs_xizxp_8z0uvwwx}"
result = atbash_cipher(text)
result


'picoCTF{atbash_crack_8a0feddc}'

### Second Technique: Stegseek and Affine

Stegseek can be used to perform a brute-force attack, utilizing a password dictionary to try and find the correct key and access the hidden information.

docker run --rm -it -v "$(pwd):/steg" rickdejager/stegseek /steg/atbash.jpg /steg/rockyou.txt

cat atbash.jpg.out 

output: krxlXGU{zgyzhs_xizxp_8z0uvwwx}

#### Affine cipher

In [70]:
# Function to calculate the modular inverse of 'a' modulo m in Z26
def modinv(a, m):
    """Calculates the modular inverse of 'a' modulo 'm', if it exists."""
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

# Function to decrypt using the affine cipher
def affine_decrypt(cipher_text, a, b):
    """
    Decrypts the given cipher text using the affine cipher with key (a, b).
    The decryption function is:
      d(y) = a_inv * (y - b) mod 26
    where a_inv is the modular inverse of 'a' in Z26.
    """
    # Calculate the modular inverse of 'a' in Z26
    a_inv = modinv(a, 26)
    if a_inv is None:
        return None  # If 'a' is not invertible, skip this attempt
    
    decrypted = ""
    for char in cipher_text:
        if char.isalpha():
            # Convert uppercase letter to number: A -> 0, B -> 1, ..., Z -> 25.
            is_upper = char.isupper()
            base = ord('A') if is_upper else ord('a')
            y = ord(char) - base
            # Apply the decryption function.
            x = (a_inv * (y - b)) % 26
            # Convert the number back to a letter.
            decrypted += chr(x + base)
        else:
            # Keep non-alphabetical characters unchanged (spaces, line breaks, etc.)
            decrypted += char
    return decrypted

# Provided cipher text
cipher_text = "krxlXGU{zgyzhs_xizxp_8z0uvwwx}"

# Test all possible combinations of (a, b) where a is invertible in Z26
valid_a_values = [a for a in range(1, 26) if modinv(a, 26) is not None]
possible_decryptions = {}

# Try all valid combinations of a and b
for a in valid_a_values:
    for b in range(26):
        decrypted_text = affine_decrypt(cipher_text, a, b)
        if decrypted_text:
            possible_decryptions[(a, b)] = decrypted_text

import pandas as pd

# Create a DataFrame with the results
df_results = pd.DataFrame(possible_decryptions.items(), columns=["(a, b)", "Decrypted Text"])

# Display the first 100 rows of the results
df_results.head(100)


,"(a, b)",Decrypted Text
0,"(1, 0)",krxlXGU{zgyzhs_xizxp_8z0uvwwx}
1,"(1, 1)",jqwkWFT{yfxygr_whywo_8y0tuvvw}
2,"(1, 2)",ipvjVES{xewxfq_vgxvn_8x0stuuv}
3,"(1, 3)",houiUDR{wdvwep_ufwum_8w0rsttu}
4,"(1, 4)",gnthTCQ{vcuvdo_tevtl_8v0qrsst}
...,...,...
95,"(7, 17)",zamoMRT{qrbqgp_mvqmw_8q0tixxm}
96,"(7, 18)",klxzXCE{bcmbra_xgbxh_8b0etiix}
97,"(7, 19)",vwikINP{mnxmcl_irmis_8m0petti}
98,"(7, 20)",ghtvTYA{xyixnw_tcxtd_8x0apeet}


Finally, when \(a = 25\) and \(b = 25\), we get a text.


In [69]:
df_results.iloc[311, 1]  


'picoCTF{atbash_crack_8a0feddc}'